In [9]:
# download the NYC Flights dataset to our workspace
import urllib

print("- Downloading NYC Flights dataset... ", end='', flush=True)
url = "https://storage.googleapis.com/dask-tutorial-data/nycflights.tar.gz"
filename, headers = urllib.request.urlretrieve(url, '../data/csv/nycflights.tar.gz')
print("Done!", flush=True)

- Downloading NYC Flights dataset... Done!


In [10]:
import tarfile
with tarfile.open(filename, mode='r:gz') as flights:
            flights.extractall('../data/csv/')

In [11]:
import os
import dask.dataframe as dd

df = dd.read_csv(os.path.join('../data/csv/', 'nycflights/', '*.csv'),
                 parse_dates={'Date': [0, 1, 2]})

df

,Date,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,Diverted
npartitions=10,,,,,,,,,,,,,,,,,,,,,
,datetime64[ns],int64,float64,int64,float64,int64,object,int64,float64,float64,int64,float64,float64,float64,object,object,float64,float64,float64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [12]:
df_train = df[['CRSDepTime',  'CRSArrTime', 'Cancelled']]
df_train.head()

,CRSDepTime,CRSArrTime,Cancelled
0,1540,1701,0
1,1540,1701,0
2,1540,1701,0
3,1540,1701,0
4,1540,1701,0


In [13]:
df_train.shape

(Delayed('int-bc738b0d-2066-4cf7-b999-8412f82ea7b4'), 3)

In [14]:
df_train.isnull().sum().compute()

CRSDepTime    0
CRSArrTime    0
Cancelled     0
dtype: int64

In [15]:
df_train

,CRSDepTime,CRSArrTime,Cancelled
npartitions=10,,,
,int64,int64,int64
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
# from skorch import NeuralNetRegressor
import torch.optim as optim
from dask.diagnostics import ProgressBar
from dask_ml.wrappers import Incremental

# HOMEWORK: Use Incremental to wrapp a torch model an run!  train and test!
class ShallowNet(nn.Module):
    def __init__(self, n_features=5):
        super().__init__()
        self.layer1 = nn.Linear(n_features, 1)

    def forward(self, x):
        return F.relu(self.layer1(x))

# sn = ShallowNet()
niceties = {
    "callbacks": False,
    "warm_start": False,
    "train_split": None,
    "max_epochs": 1,
}

model = ShallowNet(
    
    optimizer=optim.SGD,
    optimizer__lr=0.1,
    optimizer__momentum=0.9,
    batch_size=64,
    **niceties,
)

parallel_shallowNet = Incremental(model)